In [ ]:
%%writefile strands_agents_openai.py
from strands import Agent, tool
from strands_tools import calculator
import argparse
import json
from strands.models.litellm import LiteLLMModel
import os
from dotenv import load_dotenv

load_dotenv()

# Create a custom tool
@tool
def weather():
    """ Get weather """ # Dummy implementation
    return "sunny"

model = "gpt-4o-mini"
litellm_model = LiteLLMModel(
    model_id=model, params={"max_tokens": 8192, "temperature": 0.7}
)


agent = Agent(
    model=litellm_model,
    tools=[calculator, weather],
    system_prompt="You're a helpful assistant. You can do simple math calculation, and tell the weather."
)

def strands_agent_open_ai(payload):
    """
    Invoke the agent with a payload
    """
    user_input = payload.get("prompt")
    response = agent(user_input)
    return response.message['content'][0]['text']

if __name__ == "__main__":
    parser = argparse.ArgumentParser()
    parser.add_argument("payload", type=str)
    args = parser.parse_args()
    response = strands_agent_open_ai(json.loads(args.payload))
    print(response)

In [ ]:
!python strands_agents_openai.py "{\"prompt\": \"海上升明月，下一句是什么\"}"

In [ ]:
from bedrock_agentcore.services.identity import IdentityClient

from boto3.session import Session
import boto3

import os
from dotenv import load_dotenv

load_dotenv()
openai_api_key = os.getenv("OPENAI_API_KEY")

boto_session = Session()
region = boto_session.region_name

#Configure API Key Provider
identity_client = IdentityClient(region=region)

api_key_provider = identity_client.create_api_key_credential_provider({
    "name": "openai-apikey-provider",
    "apiKey": openai_api_key # Replace it with the API key you obtain from the external application vendor, e.g., OpenAI
})

In [9]:
%%writefile strands_agents_openai.py
import json
from strands import Agent, tool
from strands_tools import calculator
from strands.models.litellm import LiteLLMModel
from bedrock_agentcore.runtime import BedrockAgentCoreApp
from botocore.exceptions import ClientError
from boto3.session import Session

boto_session = Session()

secret_name = "bedrock-agentcore-identity!default/apikey/openai-apikey-provider"

def get_secret(session="", secret_name=""):

    session = session
    secret_name = secret_name
    region_name = session.region_name

    # Create a Secrets Manager client

    client = session.client(
        service_name='secretsmanager',
        region_name=region_name
    )

    try:
        get_secret_value_response = client.get_secret_value(
            SecretId=secret_name
        )
    except ClientError as e:
        # For a list of exceptions thrown, see
        # https://docs.aws.amazon.com/secretsmanager/latest/apireference/API_GetSecretValue.html
        raise e

    secret = get_secret_value_response['SecretString']

    return secret
    # Your code goes here.

api_key_value = get_secret(boto_session, secret_name)

data = json.loads(api_key_value)
api_key = data["api_key_value"]

app = BedrockAgentCoreApp()

@tool
def weather():
    """ Get weather """ # Dummy implementation
    return "sunny"

@app.entrypoint
async def strands_agent_open_ai(payload):
    """
    Invoke the agent with a payload
    """

    # Initialize agent after API key is set
    print("Initializing agent with OPENAI API KEY...")
    model = "gpt-4o-mini"
    litellm_model = LiteLLMModel(
        model_id=model,
        params={
            "max_tokens": 8192,
            "temperature": 0.7
        },
        client_args={
            "api_key": api_key
        }
    )

    agent = Agent(
        model=litellm_model,
        tools=[calculator, weather],
        system_prompt="You're a helpful assistant. You can do simple math calculation, and tell the weather."
    )
    print("Agent initialized successfully")

    user_input = payload.get("prompt")
    print(f"User input: {user_input}")

    try:
        response = agent(user_input)
        print(f"Agent response: {response}")
        return response.message['content'][0]['text']
    except Exception as e:
        print(f"Error in agent processing: {e}")
        raise

if __name__ == "__main__":
    app.run()

Overwriting strands_agents_openai.py


In [10]:
from bedrock_agentcore_starter_toolkit import Runtime
from boto3.session import Session
import boto3
import json
boto_session = Session()
region = boto_session.region_name
region

agentcore_runtime = Runtime()
agent_name="strands_agents_openai"

response = agentcore_runtime.configure(
    entrypoint="strands_agents_openai.py",
    auto_create_execution_role=True,
    auto_create_ecr=True,
    agent_name=agent_name,
    requirements_file="requirements.txt",
    region=region
)
response

Entrypoint parsed: file=D:\Code\github\Bedrock_agent\03-AgentCore-identity\04-Outbound Auth example\strands_agents_openai.py, bedrock_agentcore_name=strands_agents_openai
Configuring BedrockAgentCore agent: strands_agents_openai


⚠️  ℹ️  No container engine found (Docker/Finch/Podman not installed)
✅ Default deployment uses CodeBuild (no container engine needed)
💡 Run 'agentcore launch' for cloud-based building and deployment
💡 For local builds, install Docker, Finch, or Podman

⚠️  [WARNING] Platform mismatch: Current system is 'linux/amd64' but Bedrock AgentCore requires 'linux/arm64'.
For deployment options and workarounds, see: 
https://docs.aws.amazon.com/bedrock-agentcore/latest/devguide/getting-started-custom.html

Generated Dockerfile: D:\Code\github\Bedrock_agent\03-AgentCore-identity\04-Outbound Auth example\Dockerfile
Generated .dockerignore: D:\Code\github\Bedrock_agent\03-AgentCore-identity\04-Outbound Auth example\.dockerignore
Keeping 'strands_agents_openai' as default agent
Bedrock AgentCore configured: D:\Code\github\Bedrock_agent\03-AgentCore-identity\04-Outbound Auth example\.bedrock_agentcore.yaml


ConfigureResult(config_path=WindowsPath('D:/Code/github/Bedrock_agent/03-AgentCore-identity/04-Outbound Auth example/.bedrock_agentcore.yaml'), dockerfile_path=WindowsPath('D:/Code/github/Bedrock_agent/03-AgentCore-identity/04-Outbound Auth example/Dockerfile'), dockerignore_path=WindowsPath('D:/Code/github/Bedrock_agent/03-AgentCore-identity/04-Outbound Auth example/.dockerignore'), runtime='None', region='ap-southeast-1', account_id='566801649110', execution_role=None, ecr_repository=None, auto_create_ecr=True)

In [11]:
launch_result = agentcore_runtime.launch()
launch_result

🚀 CodeBuild mode: building in cloud (RECOMMENDED - DEFAULT)
   • Build ARM64 containers in the cloud with CodeBuild
   • No local Docker required
💡 Available deployment modes:
   • runtime.launch()                           → CodeBuild (current)
   • runtime.launch(local=True)                 → Local development
   • runtime.launch(local_build=True)           → Local build + cloud deploy (NEW)
Starting CodeBuild ARM64 deployment for agent 'strands_agents_openai' to account 566801649110 (ap-southeast-1)
Setting up AWS resources (ECR repository, execution roles)...
Getting or creating ECR repository for agent: strands_agents_openai
✅ ECR repository available: 566801649110.dkr.ecr.ap-southeast-1.amazonaws.com/bedrock-agentcore-strands_agents_openai
Getting or creating execution role for agent: strands_agents_openai
Using AWS region: ap-southeast-1, account ID: 566801649110
Role name: AmazonBedrockAgentCoreSDKRuntime-ap-southeast-1-5d0a5e0561


✅ Reusing existing ECR repository: 566801649110.dkr.ecr.ap-southeast-1.amazonaws.com/bedrock-agentcore-strands_agents_openai


✅ Reusing existing execution role: arn:aws:iam::566801649110:role/AmazonBedrockAgentCoreSDKRuntime-ap-southeast-1-5d0a5e0561
✅ Execution role available: arn:aws:iam::566801649110:role/AmazonBedrockAgentCoreSDKRuntime-ap-southeast-1-5d0a5e0561
Preparing CodeBuild project and uploading source...
Getting or creating CodeBuild execution role for agent: strands_agents_openai
Role name: AmazonBedrockAgentCoreSDKCodeBuild-ap-southeast-1-5d0a5e0561
Reusing existing CodeBuild execution role: arn:aws:iam::566801649110:role/AmazonBedrockAgentCoreSDKCodeBuild-ap-southeast-1-5d0a5e0561
Using .dockerignore with 44 patterns
Uploaded source to S3: strands_agents_openai/source.zip
Updated CodeBuild project: bedrock-agentcore-strands_agents_openai-builder
Starting CodeBuild build (this may take several minutes)...
Starting CodeBuild monitoring...
🔄 PROVISIONING started (total: 2s)
✅ PROVISIONING completed in 8.5s
🔄 DOWNLOAD_SOURCE started (total: 10s)
✅ DOWNLOAD_SOURCE completed in 2.2s
🔄 BUILD started 

LaunchResult(mode='codebuild', tag='bedrock_agentcore-strands_agents_openai:latest', env_vars=None, port=None, runtime=None, ecr_uri='566801649110.dkr.ecr.ap-southeast-1.amazonaws.com/bedrock-agentcore-strands_agents_openai', agent_id='strands_agents_openai-HwbpkMBz7a', agent_arn='arn:aws:bedrock-agentcore:ap-southeast-1:566801649110:runtime/strands_agents_openai-HwbpkMBz7a', codebuild_id='bedrock-agentcore-strands_agents_openai-builder:1b332ec2-f63d-4783-9086-19cb66b260bc', build_output=None)

In [12]:
import json
agentcore_control_client = boto3.client(
    'bedrock-agentcore-control',
    region_name=region
)

runtime_response = agentcore_control_client.get_agent_runtime(
    agentRuntimeId=launch_result.agent_id
)
runtime_role = runtime_response['roleArn']

policies_to_add = {
    "Version": "2012-10-17",
    "Statement": [
        {
            "Sid": "GetResourceAPIKey",
            "Effect": "Allow",
            "Action": [
                "bedrock-agentcore:GetResourceApiKey"
            ],
            "Resource": "*"
        },
        {
            "Sid": "SecretManager",
            "Effect": "Allow",
            "Action": [
                "secretsmanager:GetSecretValue"
            ],
            "Resource": "arn:aws:secretsmanager:*:*:secret:bedrock-agentcore*"
        }
    ]
}
iam_client = boto3.client(
    'iam',
    region_name=region
)

response = iam_client.put_role_policy(
    PolicyDocument=json.dumps(policies_to_add),
    PolicyName="outbound_policies",
    RoleName=runtime_role.split("/")[1],
)

In [13]:
import time
status_response = agentcore_runtime.status()
status = status_response.endpoint['status']
end_status = ['READY', 'CREATE_FAILED', 'DELETE_FAILED', 'UPDATE_FAILED']
while status not in end_status:
    time.sleep(10)
    status_response = agentcore_runtime.status()
    status = status_response.endpoint['status']
    print(status)
status

Retrieved Bedrock AgentCore status for: strands_agents_openai


'READY'

In [14]:
invoke_response = agentcore_runtime.invoke({"prompt": "Hello"}, user_id="userid_1234567890")
invoke_response

{'ResponseMetadata': {'RequestId': '1df3ef63-7bec-4dde-b64c-619dfbbcc9e9',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'date': 'Wed, 12 Nov 2025 03:09:58 GMT',
   'content-type': 'application/json',
   'transfer-encoding': 'chunked',
   'connection': 'keep-alive',
   'x-amzn-requestid': '1df3ef63-7bec-4dde-b64c-619dfbbcc9e9',
   'baggage': 'Self=1-6913fa83-28ac3a547103c92444b3b0cd,session.id=5b7b3082-87b5-43b5-9842-7eb930787389',
   'x-amzn-bedrock-agentcore-runtime-session-id': '5b7b3082-87b5-43b5-9842-7eb930787389',
   'x-amzn-trace-id': 'Root=1-6913fa83-6ccfccdd33794898452c1065;Parent=29467a85e32df0b8;Sampled=1;Self=1-6913fa83-28ac3a547103c92444b3b0cd'},
  'RetryAttempts': 0},
 'runtimeSessionId': '5b7b3082-87b5-43b5-9842-7eb930787389',
 'traceId': 'Root=1-6913fa83-6ccfccdd33794898452c1065;Parent=29467a85e32df0b8;Sampled=1;Self=1-6913fa83-28ac3a547103c92444b3b0cd',
 'baggage': 'Self=1-6913fa83-28ac3a547103c92444b3b0cd,session.id=5b7b3082-87b5-43b5-9842-7eb930787389',
 'contentType': 

In [ ]:
from IPython.display import Markdown, display
response_text = invoke_response['response'][0]
display(Markdown(response_text))

In [ ]:
agent_arn = launch_result.agent_arn
agentcore_client = boto3.client(
    'bedrock-agentcore',
    region_name=region
)

boto3_response = agentcore_client.invoke_agent_runtime(
    agentRuntimeArn=agent_arn,
    runtimeUserId="userid_1234567890",
    qualifier="DEFAULT",
    payload=json.dumps({"prompt": "How much is 2X2?"})
)
if "text/event-stream" in boto3_response.get("contentType", ""):
    content = []
    for line in boto3_response["response"].iter_lines(chunk_size=1):
        if line:
            line = line.decode("utf-8")
            if line.startswith("data: "):
                line = line[6:]
                content.append(line)
    display(Markdown("\n".join(content)))
else:
    try:
        events = []
        for event in boto3_response.get("response", []):
            events.append(event)
    except Exception as e:
        events = [f"Error reading EventStream: {e}"]
    display(Markdown(json.loads(events[0].decode("utf-8"))))